In [1]:
%reload_ext autoreload
%autoreload 2
if '__file__' not in globals():
    __file__, __name__ = globals()['__vsc_ipynb_file__'], '__ipynb__'
    import types, sys; sys.modules['__ipynb__'] = types.ModuleType('__ipynb__')

import sys, os
if os.path.abspath('.') not in sys.path: sys.path.append(os.path.abspath('.'))

from pathlib import Path
from glob import glob

class text_color:
    def __init__(self, *attrs): self.clrs = list(set(attrs))
    def __ror__(self, obj): return f'\33[{";".join(map(str, self.clrs))}m{str(obj)}\33[0m'
    @property
    def bg(self): return text_color( next((c+70 for c in self.clrs if 30 <= c < 38), self.clrs[0]))

bold, italic, underline, strike = [1,3,4,9]
black,red,green,yellow,blue,magenta,cyan,white = [text_color(clr) for clr in range(30,38)]

import inspect
class cout:
    def __ror__(self, obj): print(f'[{inspect.stack()[1].lineno}]'|green, obj)
    def __call__(self, *args, **kwds): print(f'[{inspect.stack()[1].lineno}]'|green, *args, **kwds)
out = cout()

from dotenv import load_dotenv
load_dotenv()

jjkim_key = os.getenv("JINA_API_JJKIM")
obj76_key = os.getenv("JINA_API_OBJECTS76")

from rich import print as pprint
from IPython.display import HTML, Markdown

In [ ]:
import shlex
import subprocess
import time

def run_web_llm(profile: str = "Default", url: str = "https://www.chatgpt.com") -> None:
    """Launch browser with sanitized inputs and wait for window activation.

    Args:
        profile (str): Browser profile directory name
        url (str): URL to open
    """
    BROWSER = "microsoft-edge-stable"

    # Sanitize inputs to prevent command injection
    safe_user = shlex.quote(profile)
    safe_url = shlex.quote(url)

    # Launch the browser
    process = subprocess.Popen([
        BROWSER,
        f"--profile-directory={safe_user}",
        "--new-tab",
        safe_url
    ])
    pid = process.pid
    print(f"Browser launched with PID: {pid}")

    # Wait for browser to start and get window ID from PID
    time.sleep(2)  # Give browser time to start

    try:
        # Get window ID associated with the PID
        result = subprocess.run(['xdotool', 'search', '--pid', str(pid)],
                              capture_output=True, text=True)
        window_ids = result.stdout.strip().split('\n')

        if window_ids and window_ids[0]:
            print(window_ids)
            window_id = window_ids[0]
            print(f"Found window ID: {window_id} for PID: {pid}")

            # Activate and raise the window
            subprocess.run(['xdotool', 'windowactivate', window_id], check=True)
            subprocess.run(['xdotool', 'windowraise', window_id], check=True)
            print(f"Window activated and raised")
        else:
            print(f"No window found for PID: {pid}")

    except subprocess.CalledProcessError as e:
        print(f"Failed to get window ID for PID {pid}: {e}")


run_web_llm()

Browser launched with PID: 3193431
Opening in existing browser session.
No window found for PID: 3193431


[3186363:3186363:0826/164033.497792:ERROR:mojo/public/cpp/bindings/lib/interface_endpoint_client.cc:732] Message 0 rejected by interface blink.mojom.WidgetHost
[3186363:3186363:0826/164421.444490:ERROR:mojo/public/cpp/bindings/lib/interface_endpoint_client.cc:732] Message 0 rejected by interface blink.mojom.WidgetHost
[3186363:3186363:0826/164533.121978:ERROR:mojo/public/cpp/bindings/lib/interface_endpoint_client.cc:732] Message 0 rejected by interface blink.mojom.WidgetHost
[3186363:3186363:0826/164657.345883:ERROR:mojo/public/cpp/bindings/lib/interface_endpoint_client.cc:732] Message 0 rejected by interface blink.mojom.WidgetHost
[3246748:1:0826/165125.603388:ERROR:components/edge_picture_in_picture/content/renderer/video_hover_overlay_agent.cc:49] Connection to VideoHoverOverlayDriver dropped
[3186363:3186363:0826/165305.575131:ERROR:mojo/public/cpp/bindings/lib/interface_endpoint_client.cc:732] Message 0 rejected by interface blink.mojom.WidgetHost
[3186363:3186363:0826/165426.4132

In [16]:
result = subprocess.run(['xdotool', 'search', '--name', '--class', 'microsoft-edge'],
                                  capture_output=True, text=True)
window_ids = result.stdout.strip().split('\n')
print(window_ids)
window_ids = window_ids or []

for i, window_id in enumerate(window_ids):
    if not window_id: continue
    # Activate and raise the window
    subprocess.run(['xdotool', 'windowactivate', window_id], check=True)
    subprocess.run(['xdotool', 'windowraise', window_id], check=True)
    print(i, f"Edge window activated (ID: {window_id})")

['54525953', '44040196', '54525955', '44040220']
0 Edge window activated (ID: 54525953)
1 Edge window activated (ID: 44040196)
2 Edge window activated (ID: 54525955)
3 Edge window activated (ID: 44040220)


XGetWindowProperty[_NET_WM_DESKTOP] failed (code=1)
XGetWindowProperty[_NET_WM_DESKTOP] failed (code=1)


In [3]:

import subprocess
import time

def get_edge_window():
    """Microsoft Edge 창을 찾고 활성화합니다."""
    try:
        # Edge 프로세스 확인
        result = subprocess.run(['pgrep', '-f', 'microsoft-edge'],
                              capture_output=True, text=True)
        if not result.stdout.strip():
            print("Microsoft Edge가 실행되지 않았습니다.")
            return False

        # Edge 창 ID 찾기
        result = subprocess.run(['xdotool', 'search', '--name', '--class', 'microsoft-edge'],
                              capture_output=True, text=True)
        window_ids = result.stdout.strip().split('\n')

        if not window_ids or not window_ids[0]:
            print("Edge 창을 찾을 수 없습니다.")
            return False

        # 첫 번째 Edge 창 활성화
        window_id = window_ids[0]
        subprocess.run(['xdotool', 'windowactivate', window_id], check=True)
        subprocess.run(['xdotool', 'windowraise', window_id], check=True)

        print(f"Edge 창 활성화됨 (ID: {window_id})")
        return True

    except subprocess.CalledProcessError as e:
        print(f"창 활성화 실패: {e}")
        return False
    except Exception as e:
        print(f"오류 발생: {e}")
        return False

# Edge 창 활성화 실행
get_edge_window()


Edge 창 활성화됨 (ID: 31460485)


True